In [1]:
import pandas as pd
import pickle as pkl
import json
import re
from datetime import datetime
from collections import Counter
import os

In [2]:
import igraph as ig

In [5]:
g = ig.Graph.Famous("Zachary")
largest = g.clusters().giant()
largest.community_spinglass()


In [7]:
import fastdtw

ModuleNotFoundError: No module named 'fastdtw'

In [2]:
os.chdir('/shared/3/projects/style-influence')

In [3]:
y = 2019
df = pd.read_csv('data/final_samples/formality_'+str(y)+'.tsv', sep='\t').fillna('')
df = df[[c for c in df.columns if not c.startswith('Unnamed:')]]
df = df.drop_duplicates(subset=['comment_id'])
df.head()

<ipython-input-3-d706bd81526c>:2: DtypeWarning: Columns (0,4,5,6,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/final_samples/formality_'+str(y)+'.tsv', sep='\t').fillna('')


,body,author,comment_id,created_utc,controversiality,score,author_flair_css_class,author_flair_text,depth,max_depth,...,parent_author,parent_id,parent_created_utc,parent_controversiality,parent_score,parent_flair_css_class,parent_flair_text,tokens,comment_id_parent,tokens_parent
0,no I used to be but now im buff like matt daemon,tofu_schmo,eq8h3wy,1559865252,0,2,,,11.0,11,...,ruttydm,eq7wnb2,1559854536.0,0.0,0.0,,,"['no', 'i', 'used', 'to', 'be', 'but', 'now', ...",eq7wnb2,"['amp', 'x200b', 'your', 'cat', 'is', 'fat', '..."
1,&amp;#x200B;\n\nYour cat is fat and you're alm...,ruttydm,eq7wnb2,1559854536,0,0,,,10.0,11,...,tofu_schmo,eq7s9y5,1559852354.0,0.0,6.0,,,"['amp', 'x200b', 'your', 'cat', 'is', 'fat', '...",eq7s9y5,"['yeah', 'it', 'is', 'super', 'dangerous', 'wh..."
2,yeah it is super dangerous which is y all vega...,tofu_schmo,eq7s9y5,1559850779,0,6,,,8.0,11,...,ruttydm,eq7p0m2,1559850697.0,0.0,-5.0,,,"['yeah', 'it', 'is', 'super', 'dangerous', 'wh...",eq7p0m2,"['it', 'shows', 'the', 'dangers', 'of', 'being..."
3,It shows the dangers of being vegan.,ruttydm,eq7p0m2,1559850697,0,-5,,,7.0,11,...,tofu_schmo,eq7ovqr,1559850632.0,0.0,7.0,,,"['it', 'shows', 'the', 'dangers', 'of', 'being...",eq7ovqr,"['no', 'ur', 'video', 'had', 'like', '7', 'peo..."
4,no ur video had like 7 people that didnt look ...,tofu_schmo,eq7ovqr,1559850632,0,7,,,6.0,11,...,ruttydm,eq7mab4,1559849307.0,0.0,-2.0,,,"['no', 'ur', 'video', 'had', 'like', '7', 'peo...",eq7mab4,"['the', 'level', 'of', 'sarcasm', 'is', 'high']"


In [3]:
df = pd.concat([pd.read_csv('data/final_samples/offensive_'+str(y)+'.tsv', sep='\t').fillna('') 
                for y in range(2019,2023)]) 


<ipython-input-3-96ede5d52db9>:1: DtypeWarning: Columns (0,4,5,6,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat([pd.read_csv('data/final_samples/offensive_'+str(y)+'.tsv', sep='\t').fillna('')
<ipython-input-3-96ede5d52db9>:1: DtypeWarning: Columns (0,4,5,6,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat([pd.read_csv('data/final_samples/offensive_'+str(y)+'.tsv', sep='\t').fillna('')
<ipython-input-3-96ede5d52db9>:1: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,10,11,12,13,14,18,19,20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat([pd.read_csv('data/final_samples/offensive_'+str(y)+'.tsv', sep='\t').fillna('')
<ipython-input-3-96ede5d52db9>:1: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,10,11,12,13,14,17,18,19,20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat([pd.read_csv('data/final_samples/offensi

In [4]:
from ast import literal_eval
def conv_list(x):
    try:
        return literal_eval(str(x))   
    except Exception as e:
        #print(e)
        return []

df.tokens = df.tokens.apply(conv_list)
df.tokens_parent = df.tokens_parent.apply(conv_list)

In [5]:
import liwc
lexicon, category_names = liwc.read_dic('/shared/3/projects/agrimaTwitter/LIWC2015_English.dic')
words_per_category = {c:[k for k,v in lexicon.items() if c in v] for c in category_names}

markers = words_per_category['function (Function Words)']
m_full = set([m for m in markers if not m.endswith('*')])
m_start = set([m for m in markers if m.endswith('*')])


def match_words(words):
    words = set(words)
    toks1 = [m for m in m_start if any(w.startswith(m[:-1]) for w in words)]
    toks2 = [m for m in m_full if m in words]
    return set([t for toks in [toks1,toks2] for t in toks])

def match_words(words):
    return [w for w in words if w in m_full or any([w.startswith(m[:-1]) for m in m_start])]


In [6]:
df['parent_which_markers'] = df['tokens_parent'].apply(match_words)
df['parent_num_markers'] = df['parent_which_markers'].apply(len)


In [7]:
df['which_markers'] = df['tokens'].apply(match_words)
df['num_markers'] = df['which_markers'].apply(len)


In [8]:
def match_parent_words(row):
    if len(row['parent_which_markers']) == 0: return []
    parent_words = set(row['parent_which_markers'])
    return [x for x in row['which_markers'] if x in parent_words]

In [9]:
df['which_parent_markers'] = df.apply(match_parent_words, axis=1)
df['num_parent_markers'] = df['which_parent_markers'].apply(len)


In [10]:
df.columns

Index(['Unnamed: 0', 'body', 'author', 'comment_id', 'created_utc',
       'controversiality', 'score', 'author_flair_css_class',
       'author_flair_text', 'depth', 'max_depth', 'subreddit', 'root_id',
       'parent_author', 'parent_id', 'parent_created_utc',
       'parent_controversiality', 'parent_score', 'parent_flair_css_class',
       'parent_flair_text', 'tokens', 'comment_id_parent', 'tokens_parent',
       'parent_which_markers', 'parent_num_markers', 'which_markers',
       'num_markers', 'which_parent_markers', 'num_parent_markers'],
      dtype='object')

In [11]:
df['num_tokens'] = df['tokens'].apply(len)
df['parent_num_tokens'] = df['tokens_parent'].apply(len)

In [ ]:
author_counts = df.groupby('author')['author'].count()
author_counts = author_counts[author_counts >= 10]
author_counts = set(author_counts.index)
sub = df[df.author.isin(author_counts)]
sub['num_parent_markers_all'] = 0
sub['parent_num_tokens_all'] = 0
pd.options.mode.chained_assignment = None 
print(datetime.now())
for aid in set(sub.author):
    sub1 = sub[sub.author==aid]
    for i in sub1.index:
        words = set(sub1.parent_which_markers[i])
        sub['num_parent_markers_all'][i] = sub1.apply(lambda row: 
                                                      len(words.intersection(set(row['which_markers']))), 
                                                      axis=1).sum()
        sub['parent_num_tokens_all'][i] = min(1, sub1.apply(lambda row: 
                                         len(words.intersection(set(row['parent_which_markers']))), 
                                         axis=1).sum())
print(datetime.now())
pd.options.mode.chained_assignment = 'warn'


In [12]:
banned = pd.read_csv('/shared/3/projects/style-influence/data/subreddit-info/banned_subreddits_utc.txt')
bsr = set(banned.Subreddit[pd.to_datetime(banned.Date_Banned).dt.year.isin([2019,2020,2021,2022])])
df['banned'] = df.subreddit.isin(bsr)

In [22]:
import pickle as pkl
formality = pd.DataFrame(pkl.load(open('data/dl-inferred-style-scores/all_scores.formality.pkl', 'rb')))
formality = formality.drop_duplicates(subset=['column_id'])

In [13]:
offensive = pd.DataFrame(pkl.load(open('data/dl-inferred-style-scores/all_scores.offensive.pkl', 'rb')))
offensive = offensive.drop_duplicates(subset=['column_id'])

In [43]:
df.shape

(7528756, 32)

In [23]:
df = pd.merge(df,
               formality[['column_id','score']].rename(columns={'score':'formality',
                                                                'column_id':'comment_id'}),
               on = 'comment_id', how='left')
df = pd.merge(df,
               formality[['column_id','score']].rename(columns={'score':'parent_formality',
                                                                'column_id':'parent_id'}),
               on = 'parent_id', how='left')

In [14]:
df = pd.merge(df,
               offensive[['column_id','score']].rename(columns={'score':'offensive',
                                                                'column_id':'comment_id'}),
               on = 'comment_id', how='left')
df = pd.merge(df,
               offensive[['column_id','score']].rename(columns={'score':'parent_offensive',
                                                                'column_id':'parent_id'}),
               on = 'parent_id', how='left')

In [15]:
df.shape

(7528756, 34)

In [47]:
df.head()

,Unnamed: 0,body,author,comment_id,created_utc,controversiality,score,author_flair_css_class,author_flair_text,depth,...,num_markers,which_parent_markers,num_parent_markers,num_tokens,parent_num_tokens,banned,formality,parent_formality,offensive,parent_offensive
0,375,I didn’t fall for his bs. Still don’t. I’m nai...,dom0515,eqlg1im,1560143724,1,0,,,6.0,...,6,[],0,15,7,False,NaN,NaN,0.070,0.033
1,376,And that's how Trump got elected....,MattSpew,eqlfuip,1560143617,0,-8,,,5.0,...,3,[],0,7,6,False,NaN,NaN,0.033,0.016
2,377,Ah. I believe everything I see.,dom0515,eqlfmq9,1560143498,0,0,,,4.0,...,3,[],0,6,18,False,NaN,NaN,0.016,0.018
3,378,It's either just the camera angle or a fake tu...,MattSpew,eqlfiic,1560143434,1,1,,,3.0,...,8,[],0,18,3,False,NaN,NaN,0.018,0.090
4,379,What’s bullshit?,dom0515,eqlfd9o,1560143359,0,-1,,,2.0,...,1,[what],1,3,8,False,NaN,NaN,0.090,0.130


In [16]:
ids = set(formality.column_id)
Counter(df.comment_id.isin(ids))

Counter({True: 7058757, False: 469999})

In [24]:
ids = set(formality.column_id)
#df = df[df.comment_id.isin(ids)]

In [21]:
ids = set(df.comment_id)
Counter(formality.column_id.isin(ids))

Counter({False: 15721603, True: 3436078})

In [22]:
# df[['root_id', 'subreddit', 'body', 'max_depth', 'depth', 'banned', 
#        'author', 'comment_id', 'created_utc', 'controversiality', 'score', 'num_tokens',
#        'parent_author', 'parent_id', 'parent_created_utc', 'parent_controversiality', 'parent_score', 'parent_num_tokens',
#        'parent_num_markers', 'num_markers', 'num_parent_markers','formality','parent_formality']].\
# to_csv('data/final_samples/formality_allmetrics_'+str(y)+'.tsv', sep='\t', index=None)



In [25]:
df[df.comment_id.isin(ids)][['root_id', 'subreddit', 'body', 'max_depth', 'depth', 'banned', 
       'author', 'comment_id', 'created_utc', 'controversiality', 'score', 'num_tokens',
       'parent_author', 'parent_id', 'parent_created_utc', 'parent_controversiality', 'parent_score', 'parent_num_tokens',
       'parent_num_markers', 'num_markers', 'num_parent_markers','formality','parent_formality','offensive','parent_offensive']].\
to_csv('data/final_samples/formality_offensive_allmetrics.tsv', sep='\t', index=None)



In [23]:
del formality, offensive

In [24]:
del df
df = pd.concat([pd.read_csv('data/final_samples/formality_allmetrics_'+str(y)+'.tsv', sep='\t').fillna('') 
                for y in range(2019,2023)]) 


<ipython-input-24-36fd8d9f9377>:2: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat([pd.read_csv('data/final_samples/formality_allmetrics_'+str(y)+'.tsv', sep='\t').fillna('')


In [25]:
df.shape

(19157572, 23)

In [26]:
df.head()

,root_id,subreddit,body,max_depth,depth,banned,author,comment_id,created_utc,controversiality,...,parent_id,parent_created_utc,parent_controversiality,parent_score,parent_num_tokens,parent_num_markers,num_markers,num_parent_markers,formality,parent_formality
0,eq7kv64,vegancirclejerk,no I used to be but now im buff like matt daemon,11.0,11.0,False,tofu_schmo,eq8h3wy,1559865252.0,0.0,...,eq7wnb2,1559854536.0,0.0,0.0,12,6,8,0,0.054,0.222
1,eq7kv64,vegancirclejerk,&amp;#x200B;\n\nYour cat is fat and you're alm...,11.0,10.0,False,ruttydm,eq7wnb2,1559854536.0,0.0,...,eq7s9y5,1559852354.0,0.0,6.0,54,30,6,2,0.222,0.018
2,eq7kv64,vegancirclejerk,yeah it is super dangerous which is y all vega...,11.0,8.0,False,tofu_schmo,eq7s9y5,1559850779.0,0.0,...,eq7p0m2,1559850697.0,0.0,-5.0,7,4,30,2,0.018,0.984
3,eq7kv64,vegancirclejerk,It shows the dangers of being vegan.,11.0,7.0,False,ruttydm,eq7p0m2,1559850697.0,0.0,...,eq7ovqr,1559850632.0,0.0,7.0,17,10,4,0,0.984,0.127
4,eq7kv64,vegancirclejerk,no ur video had like 7 people that didnt look ...,11.0,6.0,False,tofu_schmo,eq7ovqr,1559850632.0,0.0,...,eq7mab4,1559849307.0,0.0,-2.0,6,3,10,1,0.127,0.964


In [27]:
df.tail()

,root_id,subreddit,body,max_depth,depth,banned,author,comment_id,created_utc,controversiality,...,parent_id,parent_created_utc,parent_controversiality,parent_score,parent_num_tokens,parent_num_markers,num_markers,num_parent_markers,formality,parent_formality
3436073,izd8wb9,MK4GTI,Damn looks like that one isn’t available anymore,9.0,1.0,False,saddest_avenger,izefnyl,1670510603.0,0.0,...,izd8wb9,1670480630.0,0.0,2.0,45,20,3,0,0.179,0.466
3436074,izd8wb9,MK4GTI,I never had any luck with any Bluetooth odb re...,9.0,0.0,False,elpideo18,izd8wb9,1670480630.0,0.0,...,,,,,0,0,20,0,0.466,
3436075,iyt2914,monarchism,Jacobines were never really democratic.,2.0,2.0,False,Fearless-Capital-396,iyt3n7a,1670109358.0,0.0,...,iyt3emv,1670109248.0,0.0,2.0,14,5,3,0,0.355,0.178
3436076,iyt2914,monarchism,"I mean, seeing how they go full anti-democracy...",2.0,1.0,False,Lord-Belou,iyt3emv,1670109248.0,0.0,...,iyt2914,1670108720.0,0.0,1.0,8,3,5,0,0.178,0.291
3436077,iyt2914,monarchism,It's a jacobine (and possibly tankie) circlejerk.,2.0,0.0,False,Fearless-Capital-396,iyt2914,1670108720.0,0.0,...,,,,,0,0,3,0,0.291,


In [30]:
df = df[df.parent_id != '']

In [31]:
df.to_csv('data/final_samples/formality_allmetrics.tsv', sep='\t', index=None)

In [4]:
df = pd.read_csv('data/final_samples/formality_allmetrics.tsv',sep='\t').fillna('')
print(df.shape)
df.head()

(16893013, 23)


,root_id,subreddit,body,max_depth,depth,banned,author,comment_id,created_utc,controversiality,...,parent_id,parent_created_utc,parent_controversiality,parent_score,parent_num_tokens,parent_num_markers,num_markers,num_parent_markers,formality,parent_formality
0,eq7kv64,vegancirclejerk,no I used to be but now im buff like matt daemon,11.0,11.0,False,tofu_schmo,eq8h3wy,1.559865e+09,0.0,...,eq7wnb2,1.559855e+09,0.0,0.0,12,6,8,0,0.054,0.222
1,eq7kv64,vegancirclejerk,&amp;#x200B;\n\nYour cat is fat and you're alm...,11.0,10.0,False,ruttydm,eq7wnb2,1.559855e+09,0.0,...,eq7s9y5,1.559852e+09,0.0,6.0,54,30,6,2,0.222,0.018
2,eq7kv64,vegancirclejerk,yeah it is super dangerous which is y all vega...,11.0,8.0,False,tofu_schmo,eq7s9y5,1.559851e+09,0.0,...,eq7p0m2,1.559851e+09,0.0,-5.0,7,4,30,2,0.018,0.984
3,eq7kv64,vegancirclejerk,It shows the dangers of being vegan.,11.0,7.0,False,ruttydm,eq7p0m2,1.559851e+09,0.0,...,eq7ovqr,1.559851e+09,0.0,7.0,17,10,4,0,0.984,0.127
4,eq7kv64,vegancirclejerk,no ur video had like 7 people that didnt look ...,11.0,6.0,False,tofu_schmo,eq7ovqr,1.559851e+09,0.0,...,eq7mab4,1.559849e+09,0.0,-2.0,6,3,10,1,0.127,0.964


In [3]:
tenure = pkl.load(open('data/user-tenures/tenures.pkl','rb'))
print(tenure.shape)
tenure.head()

(283701013, 5)


,author,subreddit,first_post_utc,last_post_utc,tenure
0,Hampsteak,memes,1563128537,1593031288,29902751
1,Hampsteak,NoahGetTheBoat,1575014247,1584175007,9160760
2,Hampsteak,Animemes,1573211709,1584532018,11320309
3,Hampsteak,Punny,1574082431,1574082431,0
4,Hampsteak,Paladins,1566423446,1642759606,76336160


In [12]:
ids = set(df.author + ' ' + df.subreddit)
tenure_sub = tenure[(tenure.author + ' ' + tenure.subreddit).isin(ids)]
tenure_sub.shape

(3216078, 5)

In [13]:
df = pd.merge(df, tenure_sub, on=['author','subreddit'], how='left')
print(df.shape)
df.head()

(16893013, 26)


,root_id,subreddit,body,max_depth,depth,banned,author,comment_id,created_utc,controversiality,...,parent_score,parent_num_tokens,parent_num_markers,num_markers,num_parent_markers,formality,parent_formality,first_post_utc,last_post_utc,tenure
0,eq7kv64,vegancirclejerk,no I used to be but now im buff like matt daemon,11.0,11.0,False,tofu_schmo,eq8h3wy,1.559865e+09,0.0,...,0.0,12,6,8,0,0.054,0.222,1.559760e+09,1.572662e+09,12902392.0
1,eq7kv64,vegancirclejerk,&amp;#x200B;\n\nYour cat is fat and you're alm...,11.0,10.0,False,ruttydm,eq7wnb2,1.559855e+09,0.0,...,6.0,54,30,6,2,0.222,0.018,1.559849e+09,1.559855e+09,5893.0
2,eq7kv64,vegancirclejerk,yeah it is super dangerous which is y all vega...,11.0,8.0,False,tofu_schmo,eq7s9y5,1.559851e+09,0.0,...,-5.0,7,4,30,2,0.018,0.984,1.559760e+09,1.572662e+09,12902392.0
3,eq7kv64,vegancirclejerk,It shows the dangers of being vegan.,11.0,7.0,False,ruttydm,eq7p0m2,1.559851e+09,0.0,...,7.0,17,10,4,0,0.984,0.127,1.559849e+09,1.559855e+09,5893.0
4,eq7kv64,vegancirclejerk,no ur video had like 7 people that didnt look ...,11.0,6.0,False,tofu_schmo,eq7ovqr,1.559851e+09,0.0,...,-2.0,6,3,10,1,0.127,0.964,1.559760e+09,1.572662e+09,12902392.0


In [14]:
df.to_csv('data/final_samples/formality_allmetrics.tsv', sep='\t', index=None)

In [12]:
banned

,Subreddit,UTC,Date_Banned
0,FTMmeetupGW,1514788393,2018-01-01
1,smallbuttstillsmokin,1514803636,2018-01-01
2,CryptoPets,1514808996,2018-01-01
3,tradenudepics,1514809858,2018-01-01
4,a2sd4f,1514840426,2018-01-01
...,...,...,...
60356,weirdfetishbutokay,1659311031,2022-07-31
60357,needlemouseporn,1659311050,2022-07-31
60358,LEOporn,1659311666,2022-07-31
60359,hookupsdfw,1659311704,2022-07-31


In [4]:
banned = pd.read_csv('/shared/3/projects/style-influence/data/subreddit-info/banned_subreddits_utc.txt')
bsr = set(banned.Subreddit[pd.to_datetime(banned.Date_Banned).dt.year.isin([2019,2020,2021,2022])])
banned.columns = [x.lower() for x in banned.columns]
banned.head()

,subreddit,utc,date_banned
0,FTMmeetupGW,1514788393,2018-01-01
1,smallbuttstillsmokin,1514803636,2018-01-01
2,CryptoPets,1514808996,2018-01-01
3,tradenudepics,1514809858,2018-01-01
4,a2sd4f,1514840426,2018-01-01


In [5]:
tenure_banned = tenure[tenure.subreddit.isin(bsr)]
tenure_banned.shape

(2877882, 5)

In [6]:
tenure_banned = pd.merge(tenure_banned, banned, on='subreddit',how='left')

In [7]:
tenure_banned.head()

,author,subreddit,first_post_utc,last_post_utc,tenure,utc,date_banned
0,pabadacus,MakeMyCoffin,1606269061,1625008943,18739882,1645240753,2022-02-19
1,SpecialOneOnReddit,DarkHumorAndMemes,1582058324,1582058324,0,1586549289,2020-04-10
2,Macluawn,gayspiderbrothel,1641648702,1641648702,0,1649089119,2022-04-04
3,fydorm,The_Donald,1570757489,1581015485,10257996,1586528191,2020-04-10
4,fydorm,LGBDropTheT,1585273054,1585273054,0,1587875274,2020-04-26


In [8]:
Counter((tenure_banned.utc - tenure_banned.last_post_utc)/60/60/24 < 30)

Counter({False: 2259579, True: 618303})

In [9]:
tenure_banned = tenure_banned[(tenure_banned.utc - tenure_banned.last_post_utc)/60/60/24 < 30]
tenure_banned = tenure_banned[tenure_banned.author != '[deleted]']

In [10]:
authors = set(tenure_banned.author)
tenure_authors = tenure[tenure.author.isin(authors)]

In [11]:
tenure_authors = pd.merge(tenure_authors, 
                          tenure_banned[['author','subreddit','utc']], on='author', how='inner')
tenure_authors.shape

(1520765922, 7)

In [12]:
tenure_authors.head()

,author,subreddit_x,first_post_utc,last_post_utc,tenure,subreddit_y,utc
0,The-Known-One,survivor,1573953143,1623282618,49329475,survivormalebeauty,1625021889
1,The-Known-One,survivormalebeauty,1576712514,1624611279,47898765,survivormalebeauty,1625021889
2,The-Known-One,victorious,1575258227,1575278986,20759,survivormalebeauty,1625021889
3,The-Known-One,offmychest,1578991672,1586803268,7811596,survivormalebeauty,1625021889
4,The-Known-One,Edgic,1584600741,1584600741,0,survivormalebeauty,1625021889


In [15]:
tenure_authors = tenure_authors[(tenure_authors.first_post_utc > tenure_authors.utc) & 
                                (tenure_authors.first_post_utc - tenure_authors.utc < 60 * 60 * 24 * 30)]
tenure_authors.shape

(44146938, 7)

In [20]:
Counter(tenure_authors.subreddit_x != tenure_authors.subreddit_y)

Counter({True: 8957405})

In [19]:
tenure_authors = tenure_authors[tenure_authors.author!='[deleted]']
tenure_authors.shape

(8957405, 7)

In [21]:
tenure_authors.head()

,author,subreddit_x,first_post_utc,last_post_utc,tenure,subreddit_y,utc
1367290783,dleon0430,southpark,1597391356,1661370296,63978940,Shuffles_Deck,1595359092
1367290795,dleon0430,puns,1596386529,1615476712,19090183,Shuffles_Deck,1595359092
1367290801,dleon0430,beagle,1595796128,1620168335,24372207,Shuffles_Deck,1595359092
1367290844,dleon0430,dataisbeautiful,1655220701,1655220701,0,TumblrInAction,1654167743
1367290878,dleon0430,aww,1655219921,1655219921,0,TumblrInAction,1654167743


In [22]:
Counter(tenure_authors.subreddit_x)

Counter({'southpark': 445,
         'puns': 327,
         'beagle': 193,
         'dataisbeautiful': 1357,
         'aww': 2161,
         'community': 444,
         'Games': 535,
         'MapPorn': 1269,
         'ActualPublicFreakouts': 1856,
         'kotakuinaction2': 467,
         'MurderedByAOC': 500,
         'Trumpgret': 369,
         'Naruto': 181,
         'PS4': 623,
         'Michigan': 314,
         'BikiniBottomTwitter': 888,
         'DankLeft': 492,
         'GameOfThronesMemes': 136,
         'YouShouldKnow': 1020,
         'ThatsInsane': 1707,
         'Invincible': 216,
         'me_irl': 1400,
         'gifsthatkeepongiving': 528,
         'PandR': 452,
         'funnyvideos': 422,
         'NotHowGirlsWork': 600,
         'reactiongifs': 605,
         'linux': 123,
         'Target': 301,
         'TwoSentenceHorror': 666,
         'SapphoAndHerFriend': 512,
         'tennis': 239,
         'noworking': 184,
         'Frasier': 171,
         'IndianDankMemes': 453,

In [29]:
tenure_authors.columns = ['author', 'subreddit', 'first_post_utc', 'last_post_utc', 'tenure',
                          'banned_subreddit', 'banned_utc']

In [30]:
tenure_authors.to_csv('working-dir/tenure-migration.csv',index=None)

In [31]:
tenure_banned.to_csv('working-dir/tenure-active-users.csv',index=None)

In [32]:
import pytz
from datetime import datetime

#datetime.fromtimestamp(int(created_utc), pytz.UTC)

In [54]:
datetime.fromtimestamp(1610*1000000, pytz.UTC)

datetime.datetime(2021, 1, 7, 6, 13, 20, tzinfo=<UTC>)

In [17]:
df = pd.read_csv('data/final_samples/formality_allmetrics.tsv',sep='\t').fillna('')
print(df.shape)
df.head()

(16893013, 26)


,root_id,subreddit,body,max_depth,depth,banned,author,comment_id,created_utc,controversiality,...,parent_score,parent_num_tokens,parent_num_markers,num_markers,num_parent_markers,formality,parent_formality,first_post_utc,last_post_utc,tenure
0,eq7kv64,vegancirclejerk,no I used to be but now im buff like matt daemon,11.0,11.0,False,tofu_schmo,eq8h3wy,1.559865e+09,0.0,...,0.0,12,6,8,0,0.054,0.222,1559759570.0,1572661962.0,12902392.0
1,eq7kv64,vegancirclejerk,&amp;#x200B;\n\nYour cat is fat and you're alm...,11.0,10.0,False,ruttydm,eq7wnb2,1.559855e+09,0.0,...,6.0,54,30,6,2,0.222,0.018,1559848643.0,1559854536.0,5893.0
2,eq7kv64,vegancirclejerk,yeah it is super dangerous which is y all vega...,11.0,8.0,False,tofu_schmo,eq7s9y5,1.559851e+09,0.0,...,-5.0,7,4,30,2,0.018,0.984,1559759570.0,1572661962.0,12902392.0
3,eq7kv64,vegancirclejerk,It shows the dangers of being vegan.,11.0,7.0,False,ruttydm,eq7p0m2,1.559851e+09,0.0,...,7.0,17,10,4,0,0.984,0.127,1559848643.0,1559854536.0,5893.0
4,eq7kv64,vegancirclejerk,no ur video had like 7 people that didnt look ...,11.0,6.0,False,tofu_schmo,eq7ovqr,1.559851e+09,0.0,...,-2.0,6,3,10,1,0.127,0.964,1559759570.0,1572661962.0,12902392.0


In [46]:
import pytz
from datetime import datetime

def get_filename(x):
    t = datetime.fromtimestamp(int(x), pytz.UTC)
    
    y = t.year
    m = t.month
    pad = ""
    if m==12: 
        y += 1
        m = 1
    else: 
        m += 1
    if m < 10: pad = "0"
    return "RC_" + str(y) + "-" + pad + str(m) + ".gz"

print(datetime.now())
fname = df.created_utc.apply(get_filename)
print(datetime.now())


2023-06-01 12:09:45.869607
2023-06-01 12:10:27.058671


In [ ]:
set(fname)

In [57]:
set(sub2[sub2.author=='zzzzel'].subreddit)

{'Parahumans'}

,author,subreddit
0,-----------_---,"{iamatotalpieceofshit, Cringetopia}"
1,----Jay----,{teenagers}
2,---0__0---,{memes}
3,---Hollow---,{ShingekiNoKyojin}
4,--Aeo--,{transhumanism}
...,...,...
51398,zzpza,{MotoUK}
51399,zzsparkzz,"{BeforeNAfterAdoption, addiction}"
51400,zztopzz_101,{birthcontrol}
51401,zzzzel,{Parahumans}


In [103]:
sorted(os.listdir(dirname))

['RC_2005-12.gz',
 'RC_2006-01.gz',
 'RC_2006-02.gz',
 'RC_2006-03.gz',
 'RC_2006-04.gz',
 'RC_2006-05.gz',
 'RC_2006-06.gz',
 'RC_2006-07.gz',
 'RC_2006-08.gz',
 'RC_2006-09.gz',
 'RC_2006-10.gz',
 'RC_2006-11.gz',
 'RC_2006-12.gz',
 'RC_2007-01.gz',
 'RC_2007-02.gz',
 'RC_2007-03.gz',
 'RC_2007-04.gz',
 'RC_2007-05.gz',
 'RC_2007-06.gz',
 'RC_2007-07.gz',
 'RC_2007-08.gz',
 'RC_2007-09.gz',
 'RC_2007-10.gz',
 'RC_2007-11.gz',
 'RC_2007-12.gz',
 'RC_2008-01.gz',
 'RC_2008-02.gz',
 'RC_2008-03.gz',
 'RC_2008-04.gz',
 'RC_2008-05.gz',
 'RC_2008-06.gz',
 'RC_2008-07.gz',
 'RC_2008-08.gz',
 'RC_2008-09.gz',
 'RC_2008-10.gz',
 'RC_2008-11.gz',
 'RC_2008-12.gz',
 'RC_2009-01.gz',
 'RC_2009-02.gz',
 'RC_2009-03.gz',
 'RC_2009-04.gz',
 'RC_2009-05.gz',
 'RC_2009-06.gz',
 'RC_2009-07.gz',
 'RC_2009-08.gz',
 'RC_2009-09.gz',
 'RC_2009-10.gz',
 'RC_2009-11.gz',
 'RC_2009-12.gz',
 'RC_2010-01.gz',
 'RC_2010-02.gz',
 'RC_2010-03.gz',
 'RC_2010-04.gz',
 'RC_2010-05.gz',
 'RC_2010-06.gz',
 'RC_2010-

{'RC_2019-07.gz',
 'RC_2019-08.gz',
 'RC_2019-09.gz',
 'RC_2019-10.gz',
 'RC_2019-11.gz',
 'RC_2019-12.gz',
 'RC_2020-01.gz',
 'RC_2020-02.gz',
 'RC_2020-03.gz',
 'RC_2020-04.gz',
 'RC_2020-05.gz',
 'RC_2020-06.gz',
 'RC_2020-07.gz',
 'RC_2020-08.gz',
 'RC_2020-09.gz',
 'RC_2020-10.gz',
 'RC_2020-11.gz',
 'RC_2020-12.gz',
 'RC_2021-01.gz',
 'RC_2021-02.gz',
 'RC_2021-03.gz',
 'RC_2021-04.gz',
 'RC_2021-05.gz',
 'RC_2021-06.gz',
 'RC_2021-07.gz',
 'RC_2021-08.gz',
 'RC_2021-09.gz',
 'RC_2021-10.gz',
 'RC_2021-11.gz',
 'RC_2021-12.gz',
 'RC_2022-01.gz',
 'RC_2022-02.gz',
 'RC_2022-03.gz',
 'RC_2022-04.gz',
 'RC_2022-05.gz',
 'RC_2022-06.gz',
 'RC_2022-07.gz',
 'RC_2022-08.gz',
 'RC_2022-11.gz',
 'RC_2022-12.gz'}

In [132]:
import gzip
dirname = '/shared/2/datasets/reddit-dump-all/user-subreddit-counts/'

activity_df = [activity_df]
for f in ['RC_2022-09.gz','RC_2023-01.gz']:#sorted(list(set(fname).intersection(os.listdir(dirname)))):

    print(datetime.now(), f)

    sub = df[fname == f]
    f_authors = set(sub.author)
    sub = sub.groupby(['author','subreddit']).count().reset_index().\
                groupby('author')['subreddit'].agg(lambda x: set(x)).reset_index()
    f_subreddits = {sub.author[i]: sub.subreddit[i] for i in sub.index}
    f_counts = {}

    with gzip.open(dirname + f, 'r') as fle:
        ind = 0
        for line in fle:
            j = json.loads(line)
            if j['user'] in f_authors: 
                u = j['user']
                c = Counter(j['counts'])
                c = [[k, u, f, c[k], j['total_comments']] for k in f_subreddits[u]]
                f_counts[u] = c
            ind += 1

    activity_df.append(pd.concat([pd.DataFrame(c, columns=['subreddit','author','fname',
                                                          'subreddit_activity','total_activity']) 
                                  for u,c in f_counts.items()]))


activity_df = pd.concat(activity_df)

2023-06-01 20:16:56.810586 RC_2022-09.gz
2023-06-01 20:19:14.086037 RC_2023-01.gz


TypeError: cannot concatenate object of type '<class 'list'>'; only Series and DataFrame objs are valid

In [143]:
activity_df = pd.concat(activity_df)

In [145]:
activity_df.tail()

,subreddit,author,fname,subreddit_activity,total_activity
0,halo,ShadowCat117,RC_2023-01.gz,0,1
0,TwoBestFriendsPlay,Shiplord13,RC_2023-01.gz,1,1
0,raisedbynarcissists,TobyKeene,RC_2023-01.gz,0,1
1,terriblefacebookmemes,TobyKeene,RC_2023-01.gz,0,1
0,CarTalkUK,colin_staples,RC_2023-01.gz,0,1


In [146]:
activity_df.to_csv('data/user_activity.csv', index=None)

In [147]:
df['fname'] = fname

In [148]:
df.shape

(16893013, 27)

In [150]:
df = df[[c for c in df.columns if c not in ['subreddit_activity','total_activity']]]

In [152]:
temp = pd.merge(df, activity_df, on=['subreddit','author','fname'], how='left')
temp.shape

(16893013, 29)

In [153]:
fill_cols = ['subreddit_activity','total_activity']
temp[fill_cols] = temp[fill_cols].apply(lambda x: x.fillna(0))
temp.head()

,root_id,subreddit,body,max_depth,depth,banned,author,comment_id,created_utc,controversiality,...,num_markers,num_parent_markers,formality,parent_formality,first_post_utc,last_post_utc,tenure,fname,subreddit_activity,total_activity
0,eq7kv64,vegancirclejerk,no I used to be but now im buff like matt daemon,11.0,11.0,False,tofu_schmo,eq8h3wy,1.559865e+09,0.0,...,8,0,0.054,0.222,1559759570.0,1572661962.0,12902392.0,RC_2019-07.gz,8.0,101.0
1,eq7kv64,vegancirclejerk,&amp;#x200B;\n\nYour cat is fat and you're alm...,11.0,10.0,False,ruttydm,eq7wnb2,1.559855e+09,0.0,...,6,2,0.222,0.018,1559848643.0,1559854536.0,5893.0,RC_2019-07.gz,0.0,8.0
2,eq7kv64,vegancirclejerk,yeah it is super dangerous which is y all vega...,11.0,8.0,False,tofu_schmo,eq7s9y5,1.559851e+09,0.0,...,30,2,0.018,0.984,1559759570.0,1572661962.0,12902392.0,RC_2019-07.gz,8.0,101.0
3,eq7kv64,vegancirclejerk,It shows the dangers of being vegan.,11.0,7.0,False,ruttydm,eq7p0m2,1.559851e+09,0.0,...,4,0,0.984,0.127,1559848643.0,1559854536.0,5893.0,RC_2019-07.gz,0.0,8.0
4,eq7kv64,vegancirclejerk,no ur video had like 7 people that didnt look ...,11.0,6.0,False,tofu_schmo,eq7ovqr,1.559851e+09,0.0,...,10,1,0.127,0.964,1559759570.0,1572661962.0,12902392.0,RC_2019-07.gz,8.0,101.0


In [157]:
temp[[c for c in temp.columns if c not in ['fname']]].\
to_csv('data/final_samples/formality_allmetrics.tsv', sep='\t', index=None)

In [158]:
del temp

In [159]:
op = pd.read_csv('data/reddit-original-posts/01.original_posts.df.tsv', sep='\t')
print(op.shape)
op.head()

(1461424, 3)


,post_id,title,text
0,bvepl0,Can anyone guess the crossbreed of my dog?,NaN
1,bveqg6,I take it the Porn block isn't happening then....,[deleted]
2,bves61,When I'm ovulating I have strong urges to get ...,The urges go away after a couple of days. It h...
3,bveu04,"College dropout came in today, heres all the p...",NaN
4,bveu3j,Update on Financial Strategy,Just wanted to give everyone a heads up on wha...


In [163]:
import liwc
lexicon, category_names = liwc.read_dic('/shared/3/projects/agrimaTwitter/LIWC2015_English.dic')
words_per_category = {c:[k for k,v in lexicon.items() if c in v] for c in category_names}

markers = words_per_category['function (Function Words)']
m_full = set([m for m in markers if not m.endswith('*')])
m_start = set([m for m in markers if m.endswith('*')])

def match_words(words):
    words = set(words)
    toks1 = [m for m in m_start if any(w.startswith(m[:-1]) for w in words)]
    toks2 = [m for m in m_full if m in words]
    return set([t for toks in [toks1,toks2] for t in toks])

def match_words(words):
    return [w for w in words if w in m_full or any([w.startswith(m[:-1]) for m in m_start])]

from nltk.tokenize import regexp_tokenize
def tokenizebody(x):
    x = re.sub('\\bhttp.*\\b|\\[deleted\\]','',x.lower())
    return regexp_tokenize(x, '\w+')

In [164]:
op['tokens_op'] = op['text'].fillna('').astype(str).apply(tokenizebody)

In [168]:
op['op_which_markers'] = op['tokens_op'].apply(match_words)

In [169]:
op['op_num_markers'] = op['op_which_markers'].apply(len)
op['op_num_tokens'] = op['tokens_op'].apply(len)

In [174]:
op.head(20)

,post_id,title,text,tokens_op,op_which_markers,op_num_markers,op_num_tokens
0,bvepl0,Can anyone guess the crossbreed of my dog?,NaN,[],[],0,0
1,bveqg6,I take it the Porn block isn't happening then....,[deleted],[],[],0,0
2,bves61,When I'm ovulating I have strong urges to get ...,The urges go away after a couple of days. It h...,"[the, urges, go, away, after, a, couple, of, d...","[the, away, after, a, of, it, though, a, is, a...",26,46
3,bveu04,"College dropout came in today, heres all the p...",NaN,[],[],0,0
4,bveu3j,Update on Financial Strategy,Just wanted to give everyone a heads up on wha...,"[just, wanted, to, give, everyone, a, heads, u...","[just, to, everyone, a, up, on, what, to, a, l...",76,159
5,bveueo,Why DE think this is good UI design?,NaN,[],[],0,0
6,bveupr,I have a serious food addiction!,Its at least as bad as full blown acohol/nicot...,"[its, at, least, as, bad, as, full, blown, aco...","[its, at, as, as, im, it, my, or, at, somethin...",77,159
7,bvev2l,Bolin (EoB3 and B4) vs Katara,I say EoB3 and B4 for Bolin because that's whe...,"[i, say, eob3, and, b4, for, bolin, because, t...","[i, and, for, because, that, when, he, how, to...",15,40
8,bvev6r,[XB1] H: AAE Radium W: Trades,NaN,[],[],0,0
9,bvevmr,Looking for Aquascaping Advice / Thoughts,NaN,[],[],0,0


In [175]:
op.tail(20)

,post_id,title,text,tokens_op,op_which_markers,op_num_markers,op_num_tokens
1461404,mqgbi6,Can a former student [32F] ask a professor out...,[deleted],[],[],0,0
1461405,mqgbrr,"Hi!!! I’m 5’2”, I’m a tad shy, I love working ...",NaN,[],[],0,0
1461406,mqgfk1,Trading robux for bbc!,[deleted],[],[],0,0
1461407,mqgftr,Hasbro bey tournament I might record the battles,NaN,[],[],0,0
1461408,mqgjn1,Hiveswap backer update about the history of it...,NaN,[],[],0,0
1461409,mqgkbw,Nice,NaN,[],[],0,0
1461410,mqgl2x,"34 mom of 4! I’m not perfect anymore, but I ho...",[deleted],[],[],0,0
1461411,mqglnv,"Not quite p2w but equally scummy, help?",[deleted],[],[],0,0
1461412,mqgnhp,Carrying case for NT Mini Noir?,As someone who missed out on earlier NT Mini N...,"[as, someone, who, missed, out, on, earlier, n...","[as, someone, who, out, on, i, was, when, this...",56,113
1461413,mqgp94,Nailed it. It’s finally finished.,NaN,[],[],0,0


In [176]:
op.to_csv('data/reddit-original-posts/03.original_posts.function_words.tsv', sep='\t', index=None)